In [1]:
import pandas as pd

Problem 1

In [2]:
def check_column_types(left_df, right_df, left_on, right_on):
    left_types = left_df[left_on].dtypes
    right_types = right_df[right_on].dtypes

    for column in left_types.index:
        if left_types[column] != right_types[column]:
            raise ValueError(f"Column '{column}' has incompatible types in the left and right dataframes.")


In [3]:
def custom_merge(left_df, right_df, left_on, right_on, validate=False, enforce_superset=False):
    if validate:
        if enforce_superset:
            # Check if left dataframe contains a superset of merge key combinations in the right dataframe
            left_unique_keys = left_df[left_on].drop_duplicates()
            right_unique_keys = right_df[right_on].drop_duplicates()
            if not left_unique_keys.isin(right_unique_keys).all():
                raise ValueError("The left dataframe does not contain a superset of merge key combinations available in the right dataset.")
            
            # Check if right dataframe contains a superset of merge key combinations in the left dataframe
            if not right_unique_keys.isin(left_unique_keys).all():
                raise ValueError("The right dataframe does not contain a superset of merge key combinations available in the left dataset.")
        
            # Check if the two dataframes contain the same sets of merge key combinations
            if not left_unique_keys.equals(right_unique_keys):
                raise ValueError("The two dataframes do not contain the same sets of merge key combinations.")
        
        # Check merge columns for consistency of their types
        check_column_types(left_df, right_df, left_on, right_on)
    
    merged_df = pd.merge(left_df, right_df, left_on=left_on, right_on=right_on)
    
    # Determine the final set of key columns for the resulting merged dataset
    if isinstance(left_on, str) and isinstance(right_on, str):
        merged_df = merged_df.drop(columns=right_on)

    # Raise an error if there are no matched observations from the two input datasets
    if validate and merged_df.empty:
        raise ValueError("No matched observations found in the resulting dataset.")
    
    return merged_df

Problem 2

In [11]:
def calculate_overlap_days(event1_row, event2_row):
    event1_start_date, event1_end_date = pd.to_datetime(event1_row['event_1_start_date']), pd.to_datetime(event1_row['event_1_end_date'])
    event2_start_date, event2_end_date = pd.to_datetime(event2_row['event_2_start_date']), pd.to_datetime(event2_row['event_2_end_date'])
    
    # Calculate the overlap
    overlap_start = max(event1_start_date, event2_start_date)
    overlap_end = min(event1_end_date, event2_end_date)
    
    overlap_days = (overlap_end - overlap_start).days + 1
    return max(0, overlap_days)

In [12]:
# Test dataset
data = {
    'event_1_start_date': ['2023-07-10', '2023-07-15', '2023-07-20'],
    'event_1_end_date': ['2023-07-13', '2023-07-18', '2023-07-25'],
    'event_2_start_date': ['2023-07-12', '2023-07-16', '2023-07-22'],
    'event_2_end_date': ['2023-07-15', '2023-07-20', '2023-07-24'],
}

df = pd.DataFrame(data)

In [13]:
# Calculate overlap for each row in the DataFrame
df['overlap_days'] = df.apply(lambda row: calculate_overlap_days(row, df.iloc[1]), axis=1)

print(df)

  event_1_start_date event_1_end_date event_2_start_date event_2_end_date  \
0         2023-07-10       2023-07-13         2023-07-12       2023-07-15   
1         2023-07-15       2023-07-18         2023-07-16       2023-07-20   
2         2023-07-20       2023-07-25         2023-07-22       2023-07-24   

   overlap_days  
0             0  
1             3  
2             1  


Problem 3

In [14]:
def calculate_overlap_days(event_start, event_end, enrollment_start, enrollment_end):
    overlap_start = max(event_start, enrollment_start)
    overlap_end = min(event_end, enrollment_end)
    overlap_days = (overlap_end - overlap_start).days + 1
    return max(0, overlap_days)

In [16]:
# Load data from the Excel file
medical_events = pd.read_excel("all_test_data.xlsx", sheet_name="medical_events")
enrollment = pd.read_excel("all_test_data.xlsx", sheet_name="enrollment")
death_dates = pd.read_excel("all_test_data.xlsx", sheet_name="death_dates")


In [26]:
# Convert date columns to datetime objects
medical_events["even_start_date"] = pd.to_datetime(medical_events["even_start_date"])
medical_events["event_end_date"] = pd.to_datetime(medical_events["event_end_date"])
enrollment["enrollment_start_year_month"] = pd.to_datetime(enrollment["enrollment_start_year_month"], format="%Y-%m")
enrollment["enrollment_end_year_month"] = pd.to_datetime(enrollment["enrollment_end_year_month"], format="%Y-%m")


In [27]:
# Merge medical events with enrollment data based on person_id and enrollment_month
merged_data = pd.merge(medical_events, enrollment, on="patient_id")


In [28]:
# Filter out events that occurred after a person's death (if available in the person_data sheet)
if "death_dates" in death_dates.columns:
    merged_data = pd.merge(merged_data, death_dates, on="patient_id")
    merged_data = merged_data[merged_data["event_date"] <= merged_data["death_date"]]


In [29]:
# Calculate the overlap days
merged_data["overlap_days"] = merged_data.apply(lambda row: calculate_overlap_days(row["even_start_date"], row["event_end_date"], row["enrollment_start_year_month"], row["enrollment_end_year_month"]), axis=1)


In [31]:
# Calculate the total overlap days for each patient
total_overlap_days = merged_data.groupby("patient_id")["overlap_days"].sum().reset_index()


In [32]:
print(total_overlap_days)

  patient_id  overlap_days
0          A            33
1          B           142
2          C           183
3          D            62
4          E            94
